In [ ]:
#!pip install polars xlsx2csv
#!pip install xlsxwriter
#!pip install polars openpyxl
#!pip install fastexcel



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.1/165.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.2 MB/s eta 0:00:00


## Sample excel template creation with all columns

In [ ]:
import polars as pl
import subprocess

# Path to the Excel file
file_path = "/content/Excelsheet.xlsx"
csv_path = "/content/Excelsheet.csv"

# Convert Excel file (.xlsx) to CSV
subprocess.run(["xlsx2csv", file_path, csv_path])

# Read CSV into a Polars DataFrame
df = pl.read_csv(csv_path)

# Display column names
print(df.columns)



['Merchant Id', 'Store Number', 'GPR-Purchase Count', 'GPR-Purchase Amount', 'GPR-Purchase Charge Purchase Count', 'GPR-Purchase Charge Purchase Amount', 'GPR-Load Count', 'GPR-Load Amount', 'GPR-Purchase Charge Load Count', 'GPR-Purchase Charge Load Amount', 'GPR-Return Count', 'GPR-Return Amount', 'GPR-Commissions Paid Merchant Count', 'GPR-Commissions Paid Merchant Amount', 'GPR-Commissions Paid Program Owner Count', 'GPR-Commissions Paid Program Owner Amount', 'GPR-Commissions Paid FDC Count', 'GPR-Commissions Paid FDC Amount', 'GPR Liability-Commissions Disbursement Count', 'GPR Liability-Commissions Disbursement Amount', 'GPR-Liability Count', 'GPR-Liability Amount', 'GO Tag-Purchase Count', 'GO Tag-Purchase Amount', 'GO Tag-Purchase Charge Purchase Count', 'GO Tag-Purchase Charge Purchase Amount', 'GO Tag-Load Count', 'GO Tag-Load Amount', 'GO Tag-Purchase Charge Load Count', 'GO Tag-Purchase Charge Load Amount', 'GO Tag-Return Count', 'GO Tag-Return Amount', 'GO Tag-Commissions

In [ ]:
# Filter columns that contain 'count' in their name (case-insensitive)
columns_with_count = [col for col in df.columns if 'count' in col.lower()]

# Print the filtered column names
print("Columns with 'count':", columns_with_count)

# Count the number of cleaned columns
num_columns_with_count = len(columns_with_count)
print(num_columns_with_count)

# Clean the filtered columns by removing ' Count' and stripping leading/trailing spaces
cleaned_columns = [col.replace(' Count', '').strip() for col in columns_with_count]

# Print the cleaned column names
print("Cleaned columns:", cleaned_columns)

# Count the number of cleaned columns
num_cleaned_columns = len(cleaned_columns)
print(num_cleaned_columns)

Columns with 'count': ['GPR-Purchase Count', 'GPR-Purchase Charge Purchase Count', 'GPR-Load Count', 'GPR-Purchase Charge Load Count', 'GPR-Return Count', 'GPR-Commissions Paid Merchant Count', 'GPR-Commissions Paid Program Owner Count', 'GPR-Commissions Paid FDC Count', 'GPR Liability-Commissions Disbursement Count', 'GPR-Liability Count', 'GO Tag-Purchase Count', 'GO Tag-Purchase Charge Purchase Count', 'GO Tag-Load Count', 'GO Tag-Purchase Charge Load Count', 'GO Tag-Return Count', 'GO Tag-Commissions Paid Merchant Count', 'GO Tag-Commissions Paid Program Owner Count', 'GO Tag-Commissions Paid FDC Count', 'GO Tag Liability-Commissions Disbursement Count', 'GO Tag-Liability Count', 'Everywhere Card-Purchase Count', 'Everywhere Card-Purchase Charge Purchase Count', 'Everywhere Card-Load Count', 'Everywhere Card-Purchase Charge Load Count', 'Everywhere Card-Return Count', 'Everywhere Card-Commissions Paid Merchant Count', 'Everywhere Card-Commissions Paid Program Owner Count', 'Everywh

#Reading from txt files and writing into provided excel template

In [ ]:
import polars as pl
import openpyxl
import os
import xml.etree.ElementTree as ET

# Path to the existing Excel file
excel_path = "/content/Output_data.xlsx"

def process_xml(file_path):
    """Reads XML from a .txt file without a root element and parses it."""
    with open(file_path, 'r', encoding='utf-8') as file:
        xml_content = file.read()
        wrapped_xml = f"<root>{xml_content}</root>"  # Add missing root element if needed
        root = ET.fromstring(wrapped_xml)

    data_dict = {}

    for item in root.findall("ValueLinkLineItem"):
        merchant_id = item.find("merchantNumber").text if item.find("merchantNumber") is not None else "Unknown"
        store_number = item.find("altMerchantNumber").text if item.find("altMerchantNumber") is not None else "Unknown"
        category = item.find("category").text if item.find("category") is not None else "Unknown"
        amount = float(item.find("amount/FSNDollarUS").attrib["amount"]) if item.find("amount/FSNDollarUS") is not None else 0.0
        count = int(item.find("count").text) if item.find("count") is not None else 0

        key = (merchant_id, store_number)

        if key not in data_dict:
            data_dict[key] = {}

        # Aggregate duplicate categories within the same XML file
        data_dict[key][f"{category} Count"] = data_dict[key].get(f"{category} Count", 0) + count
        data_dict[key][f"{category} Amount"] = data_dict[key].get(f"{category} Amount", 0.0) + amount

    return data_dict

# Read existing Excel file to get the column structure
if os.path.exists(excel_path):
    existing_df = pl.read_excel(excel_path).fill_null(0)  # Ensure no null values
else:
    raise FileNotFoundError(f"Excel file '{excel_path}' not found.")

# Get column names from the existing file
columns = existing_df.columns

# Identify count and amount columns dynamically
count_columns = [col for col in columns if " Count" in col and col != "Total Activity Count"]
amount_columns = [col for col in columns if " Amount" in col and col != "Total Activity Amount"]

# Ensure "Count" and "Amount" pairs remain together
column_pairs = [(count_col, count_col.replace(" Count", " Amount")) for count_col in count_columns]

# Process multiple XML files and update values
file_paths = ["/content/original_1.txt", "/content/original_2.txt", "/content/original_3.txt", "/content/original_4.txt"]

aggregated_data = {}

# Process each XML file and aggregate data separately
for file_path in file_paths:
    xml_data = process_xml(file_path)

    for (merchant_id, store_number), values in xml_data.items():
        key = (merchant_id, store_number)

        if key not in aggregated_data:
            aggregated_data[key] = {col: 0 for col in columns if col not in ["Merchant Id", "Store Number"]}

        # Sum values from XML into existing merchant-store keys
        for col, value in values.items():
            if col in aggregated_data[key]:  # Only update if the column exists in the Excel template
                aggregated_data[key][col] += value

# Convert aggregated data into a Polars DataFrame, ensuring all unique rows are retained
updated_df = pl.DataFrame(
    [(key[0], key[1]) + tuple(values.get(col, 0) for col in columns[2:]) for key, values in aggregated_data.items()],
    schema=[("Merchant Id", pl.Utf8), ("Store Number", pl.Utf8)] + [(col, pl.Int64 if " Count" in col else pl.Float64) for col in columns[2:]],
    strict=False  # Allow mixed types if needed
)

# If the existing file is empty, use updated_df as the base structure
if existing_df.height == 0:
    merged_df = updated_df
else:
    # Merge using "outer" to retain all unique Merchant Id & Store Numbers
    merged_df = existing_df.join(updated_df, on=["Merchant Id", "Store Number"], how="outer").fill_null(0)

# Calculate Total Activity Count & Amount
merged_df = merged_df.with_columns(
    pl.sum_horizontal(count_columns).alias("Total Activity Count").cast(pl.Int64),
    pl.sum_horizontal(amount_columns).alias("Total Activity Amount").cast(pl.Float64)
)

# Ensure output column order matches input file
merged_df = merged_df.select(columns)  # Preserve column order
# Write DataFrame to Excel (default sheet = "Sheet1")
merged_df.write_excel(excel_path)

# Rename "Sheet1" to "xml_data"
wb = openpyxl.load_workbook(excel_path)
if "Sheet1" in wb.sheetnames:
    ws = wb["Sheet1"]
    ws.title = "xml_data"  # Rename the sheet
wb.save(excel_path)
wb.close()

<ipython-input-49-599a677b7fbe>:73: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  updated_df = pl.DataFrame(


## Alternative to keep xml_data as tab name and keep the formatting away with pandas

In [ ]:
### Alternative to keep the sheet name as xml_data
# Save the updated DataFrame back to the same Excel file
import pandas as pd

# Convert Polars DataFrame to Pandas
merged_df_pandas = merged_df.to_pandas()

# Write to Excel using Pandas to retain sheet structure
with pd.ExcelWriter(excel_path, mode="a", if_sheet_exists="overlay", engine="openpyxl") as writer:
    merged_df_pandas.to_excel(writer, sheet_name="xml_data", index=False)

## Read the Output Excel

In [ ]:
# Read the updated Excel file into a Polars DataFrame
final_df = pl.read_excel(excel_path)

# Print the full DataFrame without truncation
print(final_df)


shape: (4, 72)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ Merchant  ┆ Store     ┆ GPR-Purch ┆ GPR-Purch ┆ … ┆ Not       ┆ Not       ┆ Total     ┆ Total    │
│ Id        ┆ Number    ┆ ase Count ┆ ase       ┆   ┆ Settled   ┆ Settled   ┆ Activity  ┆ Activity │
│ ---       ┆ ---       ┆ ---       ┆ Amount    ┆   ┆ Count     ┆ Amount    ┆ Count     ┆ Amount   │
│ str       ┆ str       ┆ i64       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
│           ┆           ┆           ┆ i64       ┆   ┆ i64       ┆ i64       ┆ i64       ┆ i64      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ WMUSPAYLO ┆ 000004541 ┆ 0         ┆ 0         ┆ … ┆ 0         ┆ 0         ┆ 4         ┆ -12300   │
│ CS        ┆ 32        ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ WMUSPAYLO ┆ 000002452 ┆ 0         ┆ 0         ┆ … ┆ 0         ┆ 0         